# Project 1
**Maddy Bursell, Kim McKeever**

In [36]:
#Libraries
import pandas as pd
import numpy as np
import functools
import warnings
import math
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

<class 'ModuleNotFoundError'>: No module named 'seaborn'

## Introduction

In this project, we will be reading in files that use US census data from the years 1987-2006. The goal is eventually to determine wether a *simple* or a *multiple* linear regression models might better predict enrollment in public schools. 

To achieve this goal, we first need to read in the data and parse it into a format we can use. We will do this using a series of functions for data processing and combining data. We are interested in seeing if certain variables like year and census geographic division can predict the number of students enrolled in public school. Later, we will use the data to make a simple linear regression model and a multiple linear regression model. Finally, we will use cross-validation and estimate the Mean-squared error (MSE) to see which model works better for predicting enrollment. 

## Part 1: Data Processing

### Read in the data

First, we read in the initial census data to examine using `pd.read_csv()`:

In [35]:
import_data = pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv")
import_data.head()

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

We can see that the data contains many columns and many different kinds of variables within columns. It will be helpful to clean up the data to be more understandable before we begin to use it.

### Create functions to clean and process the data

Functions can be extremely useful when you want to process multiple files in the same way. In this section, we will define 5 different functions that will be called in one wrapper function. All of the functions will work to select only the specific columns we want, create new columns we need, and format the data how we want it. 

In **census_colnames** we first want to clean up the data by selecting only certain columns and renaming one column. Our step one will be to rename the "Area_name" column as "area_name" and to grab all of the columns that end in the letter "D." To do this, it is easiest to create a function that we can use on other datasets down the line. In this function, we take in a dataframe as the input and we have an optional argument to name the new column. The default name of the new column is "enrollment." 

We only want to maintain a few columns with specific parameters: we want 'Area_name', 'STCOU', and any column that ends with the letter 'D'. Since we want to select all columns ending in "D", we are locating those names first. Creating a list of desired columns for our modified data frame by using specific coumn names from the import data and by unpacking the list of column names that end with "D" that was generated in the previous line. Next, using the `.loc()` method and the desired columns found in 'col_list', we add those columns to our dataframe. Lastly, we will convert our data into long format where each row has only one enrollment value for the column area_name. This will help to get rid of multiple observations in a given row.

In [3]:
def census_colnames(df_1, col_name = "enrollment"):
    
    # Change the column name
    df_2 = df_1.rename(columns = {"Area_name":"area_name"}, inplace = True)
    
    # Create a list of the column names and an empty list to grab all of the columns ending in "D"
    cols=list(df_1)
    col_names=[]
    
    # We will use a for loop to iterate through the columns and grab the column names ending in "D" into a list
    for x in cols:
        if x.endswith("D"):
            col_names.append(x)
        else:
            continue
            
    # We can subset our original dataframe to grab the columns we want with the list "col_list"
    col_list=['area_name', 'STCOU', *col_names]
    df_2=df_1.loc[:,col_list]
    
    # Convert the dataframe to long format using pd.melt()
    df_2 = df_2.melt(id_vars = ["area_name", "STCOU"], var_name = "info", value_name = col_name)
    
    return(df_2)
    

In **parse_year**, we are taking the output of *census_colnames* and processing it even more. Each variable in the info column holds important information. The last two characters before the "D" represent the year. We want to create a new column called "year" that stores the year. The first three characters represent the survey and the next four represent the type of value you have from the survey. We want to capture those first seven characters in another column called "measurement."

First, we will create new empty columns in our dataframe. Then, we will loop through the dataframe by the row, find the info column value, and parse out the measurment and the year. The value for each row will be added to the new columns of the dataframe. 

In [4]:
def parse_year(df_2): 
    # Create empty columns for year and measurement
    df_2["year"] = np.nan
    df_2["measurement"] = np.nan
    
    # Make a loop to grab the first 7 elements for each row in the info column and put it in the new 
    # measurement column
    for row in range(len(df_2)):
        thing = df_2.loc[row, "info"]
        measure = thing[0:7]
        
        # We are pulling out the two-digit year numbers and making it a four-digit year
        if int(thing[7:9])>=23:
            yr = "19" + str(thing[7:9])
        else:
            yr= '20'+ str(thing[7:9])
        df_2.loc[row,"year"] = int(yr)
        df_2.loc[row,"measurement"] = str(measure)
        
    return(df_2)

In the **parse_state** function, we are only editing the county-level data. This function will be called within a later function (*make_census_dfs*) once we have split our original dataframe into two new dataframes called county_df and non_county_df. In *parse_state*, we are taking the county_df and creating a new column in it where we have the state for each row. 

In [5]:
def parse_state(dataframe):
    
    # Making a new column called state that stores the last two characters of area_name
    dataframe["State"] = dataframe.loc[:,"area_name"].apply(lambda x: x[-2:])
    
    return(dataframe)

In **parse_division**, we are editing only the state-level data. This function will be called in make_census_dfs, after we have created the two new dataframes called county_df and non_county_df. In this function, we will be creating a new column in non_county_df that includes the census division that each of the states falls into. We do this by using a function within a function called *return_div*. This will check if the state is in a certain list, and return the census division to the new column in the dataframe using np.vectorize(). 

In [6]:
def parse_division(dataframe):
    
    # This function checks for each state, which division it falls into by looking for the state in a list
    def return_div(val):
        div1 = ["CONNECTICUT", 'MAINE','MASSACHUSETTS','NEW HAMPSHIRE','RHODE ISLAND','VERMONT']
        div2 = ["NEW JERSEY",'NEW YORK','PENNSYLVANIA']
        div3 = ['ILLINOIS','INDIANA','MICHIGAN','OHIO','WISCONSIN']
        div4 = ['IOWA','KANSAS','MINNESOTA','MISSOURI','NEBRASKA','NORTH DAKOTA','SOUTH DAKOTA']
        div5 = ['DELAWARE','FLORIDA','GEORGIA','MARYLAND','NORTH CAROLINA','SOUTH CAROLINA','VIRGINIA',
                'DISTRICT OF COLUMBIA','WEST VIRGINIA']
        div6 = ['ALABAMA','KENTUCKY','MISSISSIPPI','TENNESSEE']
        div7 = ['ARKANSAS','LOUISIANA','OKLAHOMA','TEXAS']
        div8 = ['ARIZONA','COLORADO','IDAHO','NEVADA','MONTANA','NEW MEXICO','UTAH','WYOMING']
        div9 = ['ALASKA','CALIFORNIA','HAWAII','OREGON','WASHINGTON']
        val = val.upper()
    
        if val in div1:
            return("Division 1")
        elif val in div2:
            return("Division 2")
        elif val in div3:
            return("Division 3")
        elif val in div4:
            return("Division 4")
        elif val in div5:
            return("Division 5")
        elif val in div6:
            return("Division 6")
        elif val in div7:
            return("Division 7")
        elif val in div8:
            return("Division 8")
        elif val in div9:
            return("Division 9")
        else:
            return("ERROR")
        
    dataframe["division"] = np.vectorize(return_div)(dataframe["area_name"])
    return(dataframe)
    
    

In **make_census_dfs**, we are processing the output from *parse_year* and creating two new dataframes. We can see that the "area_name" column holds two different kinds of values: states and other (ex: "ALABAMA" or "UNITED STATES") and counties of states (ex: "Baldwin, AL"). 

We would like to create two separate dataframes that hold the non-county-level data and the county-level data. To do that, we will create an indexing vector using a lambda function. The lambda function is searching the string within each row, in the 'area_name' column, searching to match if the fourth character of from the end is a `','`. 

This information is passed into a new column on the data frame that expresses a boolean to identify whether or not the row represented is part of a county or not. 

Now that there is a new column in the dataframe to index whether or not an area represents a county or not, the two different dataframes are created. The county data frame is generated using `.loc()` and the non_county dataframe is generated by running `np.logical_not()` method from numpy.

The State information was then pulled from the `area_name` column to be held in its own unique column called "State" in the county df. 

Once we have the two new dataframes, we will call **parse_state** on county_df which will add a column called "state" specifying the state in each row. We will also call **parse_division** on non_county_df which will add a new column in that dataframe that specifies the division of each state. 

In [7]:
def make_census_dfs(dataframe):
    
    # Making a new column to index whether or not the values in a row are counties or other (non_county)
    dataframe["is_county"] = dataframe["area_name"].apply(lambda x: x[-4] == ",")
    
    # Used the new column to subset our original dataframe into two new dataframes
    county_df = dataframe.loc[dataframe["is_county"]]
    non_county_df = dataframe.loc[np.logical_not(dataframe["is_county"])]
    
    # Called some of our functions on the county data
    county_df = parse_state(county_df)
    final_county_df = county_df[["area_name", "STCOU", "enrollment", "year", "measurement", "State"]]
    
    # Called some of our functions on the non_county data
    non_county_df = parse_division(non_county_df)
    final_non_county_df = non_county_df[["area_name", "STCOU", "enrollment", "year", "measurement", "division"]]
    
    # We returned only the final versions with the columns we want to keep
    return(final_county_df, final_non_county_df)

### Create a wrapper function that will call all of our previously defined functions

If we have several functions that function continuously, meaning the output of one function becomes the input of the next function, we can easily run all of these functions at once by putting them within another function. This saves time and energy, and keeps you from making mistakes while typing in the input for each individual function. In our wrapper function, we give the argument for the data to read in and then all of the functions will run, leaving us with our desired output. Because the wrapper function combines all of our data processing functions, we have named it "census_data_processing."

In [8]:
def census_data_processing(URL, col_name = "enrollment"):
    
    # Calling all of our data processing functions (some of them are functions within functions)
    import_data = pd.read_csv(URL)
    census_df = census_colnames(import_data)
    census_df_2 = parse_year(census_df)
    final_county_df, final_non_county_df = make_census_dfs(census_df_2)
    
    return([final_county_df, final_non_county_df])

We named the output from the first URL "file_1_list." We called the wrapper function and used the URL as the argument. The output of our function is a list of the two dataframes. The first dataframe includes county data. The second dataframe includes non-county data. 

In [9]:
file_1_list = census_data_processing("https://www4.stat.ncsu.edu/~online/datasets/EDU01a.csv")
file_1_list

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

We named the output from our second URL "file_2_list." 

In [10]:
file_2_list = census_data_processing("https://www4.stat.ncsu.edu/~online/datasets/EDU01b.csv")
file_2_list

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

## Part 2: Combine Your Data

### Combine county and state data from multiple files

We want to combine our county dataframes non-county dataframes from two separate URLs. To do this, we wrote a function that takes unlimited positional arguments of the list output from our *census_data_processing* function. 

In [11]:
def combine_data(*files):
    
    # Use lambda function to select the county and non-county dataframes from the list input
    county_data = map(lambda x: x[0], files)
    non_county_data = map(lambda x: x[1], files)
    
    # Use lambda function to concatenate the county and non-county data from multiple input file lists
    combine_county = functools.reduce(lambda x,y: pd.concat([x,y]), county_data)
    combine_non_county = functools.reduce(lambda x,y: pd.concat([x,y]), non_county_data)
    
    return([combine_county, combine_non_county])

We call our *combine_data* function on the two file lists we produced from our *census_data_processing* function. 

In [12]:
combined_enrollment = combine_data(file_1_list, file_2_list)
combined_enrollment

<class 'NameError'>: name 'file_1_list' is not defined

### Check if it generalizes

We have seen that our data processing functions work on the files from URLs labeled "EDU01a" and "EDU01b", now we will check to see if our functions are generalized enough to work on multiple new files. We will import 4 new files using their URLs. 

Let's begin by running our wrapping function *census_data_processing* on the four new files to create 4 lists of county and non-county dataframes. We will name these additional files "test" so that we can distinguish them from the main two files we have been working with. 

In [13]:
test_1 = census_data_processing("https://www4.stat.ncsu.edu/~online/datasets/PST01a.csv")
test_1

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

In [14]:
test_2 = census_data_processing("https://www4.stat.ncsu.edu/~online/datasets/PST01b.csv")
test_2

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

In [15]:
test_3 = census_data_processing("https://www4.stat.ncsu.edu/~online/datasets/PST01c.csv")
test_3

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

In [16]:
test_4 = census_data_processing("https://www4.stat.ncsu.edu/~online/datasets/PST01d.csv")
test_4

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

Now that we have the 4 lists of county and non-county dataframes from each of the 4 test datasets, we can combine all of the county data into one dataframe and combine all of the non-county data into one dataframe using our *combine_data* function.

In [17]:
test_combined_list = combine_data(test_1, test_2, test_3, test_4)

<class 'NameError'>: name 'test_1' is not defined

In [18]:
test_combined_list

<class 'NameError'>: name 'test_combined_list' is not defined

## Part 3: Cross-Validation

### Subset the data

In the next section, we will analyze our data. We want to look only at the non-county data, so we will subset our list of dataframes that we got as output from the combine_data function we made. That function returns a list with the combined county dataframe as the first element and the combined non-county dataframe as the second element. Because list indices start at 0, we will subset the second element using [1]. We will name this new variable **combined_non_county** so we know that it holds the combined dataframes. 

In [19]:
combined_non_county = combined_enrollment[1]

<class 'NameError'>: name 'combined_enrollment' is not defined

In [20]:
combined_non_county

<class 'NameError'>: name 'combined_non_county' is not defined

Now that we just have the non-county data, we also want to remove any rows where there is "ERROR" in the division column. This means that we will only be looking at states. We will subset our dataframe by locating the rows where the division column values do not equal "ERROR." We will rename this dataframe to **combined_state** because we are now dealing with just states and not all non-county data. 

In [21]:
combined_state = combined_non_county.loc[combined_non_county["division"] != "ERROR",["enrollment", 
                                                                                     "year", "division"]]
combined_state

<class 'NameError'>: name 'combined_non_county' is not defined

### Cross-validation

In this section, we will be creating a SLR model using **year** (X) to predict **enrollment** (y), and a MLR model using **year** (X_1) and **division** (X_2) to predict **enrollment** (y).

To do this, we are creating a function to take an input of defined predictor and response variables, and the last year to be considered. This function will use certain years to fit the data and then predict the next year enrollment. We will start by training the model on the first three years to predict the fourth year. Then, we will train on four years and predict the fifth year, cycling through until we have predicted the last year included in the dataset. The output of the function will be the mean square error of the predictions for the next years. This function is called **mse_LR**. 

We want this because we are going to test if a simple linear regression model or a multiple linear regression model fits the data better by calulating MSE. Mean-squared error (MSE) measures the amount of error in our statistical models.

To do MLR, we will need to create the dummy variables. We will do this using the **division** column. We will only include the first 8 variables, leaving off "division 9" because it is redundant given all others. We use the pd.get_dummies() method to create the dummy variables.

In [22]:
pd.get_dummies(data = combined_state["division"])

<class 'NameError'>: name 'combined_state' is not defined

In [23]:
combined_state[["dummy_div_1", "dummy_div_2","dummy_div_3",
                "dummy_div_4", "dummy_div_5", "dummy_div_6", "dummy_div_7", 
                "dummy_div_8"]]= pd.get_dummies(data = combined_state["division"])[["Division 1",
                                                                                    "Division 2", 
                                                                                    "Division 3", 
                                                                                    "Division 4", 
                                                                                    "Division 5", 
                                                                                    "Division 6", 
                                                                                    "Division 7", 
                                                                                    "Division 8"]]

<class 'NameError'>: name 'combined_state' is not defined

Next, we have to define the predictors and response variables for both SLR and MLR that we will use an input for our function. 

In [24]:
SLR_predictors = combined_state[["year"]]
MLR_predictors = combined_state[["year", "dummy_div_1", "dummy_div_2", "dummy_div_3", 
                                 "dummy_div_4", "dummy_div_5", "dummy_div_6", "dummy_div_7", "dummy_div_8"]]
both_response = combined_state[["enrollment"]]

<class 'NameError'>: name 'combined_state' is not defined

In [25]:
def mse_LR(predictors, response, last_year):
    
    # We define which years we will use to train the data and which we will use to test the data
    x_train=predictors.loc[predictors['year'] <=(last_year)]
    x_test=predictors.loc[predictors['year']==last_year+1]
    y_train = response.loc[predictors['year'] <=(last_year)]
    y_test = response.loc[predictors['year']==last_year+1]
    
    # We fit the data to the model and calculate MSE for the prediction
    reg=linear_model.LinearRegression()
    reg.fit(X=x_train, y=y_train)
    preds = reg.predict(x_test)
    MSE = mean_squared_error(y_test,preds)
    
    return(round(MSE,2))

Our first function outputs the sequential MSE for each step we are training and predicting. However, we want to know the total MSE of predicting all the years after the 3rd year. To calculate the total MSE, we created a function called **sum_MSE** that will call our **mse_LR** within it and sum the MSE calculated for each step. This function will take in the predictors, response, and the first_year.

To see all of the years that are in the **year** column of the combined_state dataframe, we can use the .unique() method to print out all of the unique values in that column. Doing so we can see that the first year in the dataset or the earliest year is 1987. However, for our mse_LR function, we want our first year to be 1989 because we are using a minimum of the first 3 years to train our model. To make sure we put in the right year, we put an if statement into our function that asks if the year is less than 1989. If it is, we return an error statement. If this year is 1989 or greater, we can calculate MSE. We will test both of these things below. 

In [26]:
def sum_MSE(predictors, response, first_year):
    
    # Initialize MSE at zero
    MSE = 0
    
    # If the input year provides less than 3 years of data, return an error
    if first_year < 1989.0:
        print("Error: you have not provided more than three years to calculate MSE") 
    
    # If the input year does provide 3 years of data, we will use it as input into mse_LR to calculate the MSE 
    # and sum it
    else:
        years_list = list(combined_state.year.unique())
        years_list_2 = [item for item in years_list if item >= first_year]
        
        for year in years_list_2[0:-1]:
            MSE += mse_LR(predictors, response, year)
            
    return("MSE:", round(MSE,2))

Now that we have defined both of our functions, we can call them to calculate the MSE for our simple linear regression model (SLR_MSE) and our multuple linear regression model (MLR_MSE). 

To test whether our function will produce errors when we give it a first_year that is within the first 3 years, we will call it and give it 1987 as the first_year argument. We can see that it gives an error back, so we will give it a different first_year that includes the first 3 years of data.

In [27]:
sum_MSE(SLR_predictors, both_response, first_year = 1987)

<class 'NameError'>: name 'SLR_predictors' is not defined

In [28]:
SLR_MSE = sum_MSE(SLR_predictors, both_response, first_year = 1989)
SLR_MSE

<class 'NameError'>: name 'SLR_predictors' is not defined

In [29]:
MLR_MSE = sum_MSE(MLR_predictors, both_response, first_year = 1989)
MLR_MSE

<class 'NameError'>: name 'MLR_predictors' is not defined

As we can see from our outputs above, there is a greater amount of error with our simple linear regression model. Our multiple linear regression model has a slighlty lower error. This may be due to the fact that we are using both year **and** division to predict enrollment. More variables to predict enrollment works better than using only the year variable. 

In [30]:
import seaborn as sns
sns.regplot(x =combined_non_county['year'], y = combined_non_county["enrollment"], scatter_kws={'s':2})


<class 'ModuleNotFoundError'>: No module named 'seaborn'